In [2]:
%matplotlib inline

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use('ggplot')

import os, sys, shutil
sys.path.append("..")
import pandas as pd

from lammpsrun import LAMMPS, Prism
from PBSJobSubmissionScript import PBS_Submitter, qstat_monitor
from utility_scripts import *

from ase import Atoms, units
from ase.visualize import view
from ase.visualize.plot import plot_atoms
from ase.io.trajectory import Trajectory
from ase.io import write, read
from ase.build import molecule, bcc110, bcc111
from ase.spacegroup import crystal
from ase.data import atomic_numbers, atomic_names, atomic_masses, covalent_radii

cx1_out = r"/rds/general/user/yx6015/home/cx1_out/"
ephemeral = r"/rds/general/user/yx6015/ephemeral/"
sim_path = r"/rds/general/user/yx6015/home/LAMMPS_Simulation/HPC_Jupyter/"
print(cx1_out + '\n' + sim_path)
potential = "fe_o_water_fe_o_c_full_water.reax"

/rds/general/user/yx6015/home/cx1_out/
/rds/general/user/yx6015/home/LAMMPS_Simulation/HPC_Jupyter/


In [2]:
np.show_config()

mkl_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/rds/general/user/yx6015/home/anaconda3/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/rds/general/user/yx6015/home/anaconda3/include']
blas_mkl_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/rds/general/user/yx6015/home/anaconda3/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/rds/general/user/yx6015/home/anaconda3/include']
blas_opt_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/rds/general/user/yx6015/home/anaconda3/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/rds/general/user/yx6015/home/anaconda3/include']
lapack_mkl_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/rds/general/user/yx6015/home/anaconda3/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/rds/general/user/yx6015/

# MKL Test

In [6]:
import psutil
psutil.cpu_count(logical=False)

20

In [9]:
file_name = 'Aq_efield_0MVcm'
lammps_traj_file = '/rds/general/user/yx6015/ephemeral/2466596.cx1/Aq_efield_0MVcm.lammpstrj'

In [ ]:
mol_file = file_name + ".extxyz"
ase_traj_file = file_name + ".traj"

calc = LAMMPS()
mol = read(mol_file)
mol.set_calculator(calc)
calc.atoms = mol
calc.prism = Prism(mol.get_cell())

print("Writing ASE trajectory to ", ase_traj_file)
calc.trajectory_out = Trajectory(ase_traj_file, 'w')

print("Reading LAMMPS Trajectory from", lammps_traj_file)
calc.read_lammps_trj(lammps_trj = lammps_traj_file)
calc.trajectory_out.close()

In [1]:
#yay it works

# Conversion Speed Test

In [41]:
aq_out = '2466596.cx1'
aq_name = 'Aq_efield_0MVcm'
out_dir = os.path.join(ephemeral, aq_out)
convert_scripts_dir = r"/rds/general/user/yx6015/home/LAMMPS_Simulation/HPC_Scripts/"

In [55]:
test_list = ['', '_nocopy', '_nocopy_nosend', '_fast', '_nocopy_nosend_fast']
script_list = ["convert_script{}.py".format(x) for x in test_list]
test_args = ['{}'.format(aq_name),
             '{} {}'.format(aq_name, os.path.join(out_dir, aq_name+'.lammpstrj')),
             '{} {} {}'.format(aq_name, os.path.join(out_dir, aq_name+'.lammpstrj'), os.path.join(out_dir, aq_name+'.traj')),
             '{}'.format(aq_name),
             '{} {} {}'.format(aq_name, os.path.join(out_dir, aq_name+'.lammpstrj'), os.path.join(out_dir, aq_name+'.traj'))]
test_commands = [["python {} {}".format(x[0], x[1])] for x in zip(script_list, test_args)]

In [78]:
test_source_files = [[os.path.join(out_dir, "*extxyz"), 
                      os.path.join(convert_scripts_dir, x),
                      os.path.join(convert_scripts_dir, "lammpsrun.py")] for x in script_list]
test_source_files[0] += [os.path.join(out_dir, "*lammpstrj")]
test_source_files[3] += [os.path.join(out_dir, "*lammpstrj")]
test_names = ['convert{}'.format(x) for x in test_list]

In [57]:
convert_PBS = PBS_Submitter(job_names=test_names,
                            job_commands=test_commands,
                            modules=["mpi", "anaconda3/personal"],
                            walltime="24:00:00",
                            proc_nodes=1,
                            proc_cpus=8,
                            proc_mpiprocs=8, 
                            memory=32,
                            source_files=test_source_files)

In [58]:
convert_out, convert_err = convert_PBS.run()
print(convert_out,'\n', test_names)

['2474193.cx1', '2474194.cx1', '2474195.cx1', '2474196.cx1', '2474197.cx1'] 
 ['convert', 'convert_nocopy', 'convert_nocopy_nosend', 'convert_fast', 'convert_nocopy_nosend_fast']


In [61]:
convert_PBS.params

{'source_files': [['/rds/general/user/yx6015/ephemeral/2466596.cx1/*extxyz',
   '/rds/general/user/yx6015/home/LAMMPS_Simulation/HPC_Scripts/convert_script.py',
   '/rds/general/user/yx6015/home/LAMMPS_Simulation/HPC_Scripts/lammpsrun.py',
   '/rds/general/user/yx6015/ephemeral/2466596.cx1/*lammpstrj'],
  ['/rds/general/user/yx6015/ephemeral/2466596.cx1/*extxyz',
   '/rds/general/user/yx6015/home/LAMMPS_Simulation/HPC_Scripts/convert_script_nocopy.py',
   '/rds/general/user/yx6015/home/LAMMPS_Simulation/HPC_Scripts/lammpsrun.py'],
  ['/rds/general/user/yx6015/ephemeral/2466596.cx1/*extxyz',
   '/rds/general/user/yx6015/home/LAMMPS_Simulation/HPC_Scripts/convert_script_nocopy_nosend.py',
   '/rds/general/user/yx6015/home/LAMMPS_Simulation/HPC_Scripts/lammpsrun.py'],
  ['/rds/general/user/yx6015/ephemeral/2466596.cx1/*extxyz',
   '/rds/general/user/yx6015/home/LAMMPS_Simulation/HPC_Scripts/convert_script_fast.py',
   '/rds/general/user/yx6015/home/LAMMPS_Simulation/HPC_Scripts/lammpsrun.